# DATA PREPROCESSING

Initially, the data is preprocessed to create a clean and complete dataset for analysis. This involves standardizing formats, completing missing records and addressing any inconsistencies to ensure the reliability of subsequent analyses.

For this make sure to have a folder named **data** within this repository where the following csv files should appear:
- accidents_original.csv
- weather2018.csv
- borough_area.csv
- zip_code_area.csv



In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read CSV file
df = pd.read_csv('data/accidents_original.csv')
print(df.columns)

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')


First we are going to substract columns which do not provide information relevant for our analysis, which at first mainly include all contributing factors except for car 1, other vehicles type but 1 and redundant location information.

In [3]:
df.drop(columns=['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'CONTRIBUTING FACTOR VEHICLE 2', 
                 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 
                 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5', 
                 'CROSS STREET NAME', 'OFF STREET NAME', 'LOCATION', 'COLLISION_ID', 'ON STREET NAME'], inplace=True)

In [4]:
df.rename(columns={'VEHICLE TYPE CODE 1': 'VEHICLE TYPE'}, inplace=True)
df.rename(columns={'CONTRIBUTING FACTOR VEHICLE 1': 'CONTRIBUTING FACTOR'}, inplace=True)

Now, since some of the cars have been incorrectly typen and there are over 217 classes, we are going to do clusters for car types

In [5]:
print(df['VEHICLE TYPE'].value_counts())

VEHICLE TYPE
Sedan                                  36536
Station Wagon/Sport Utility Vehicle    28200
Taxi                                    3818
Pick-up Truck                           2479
Box Truck                               1678
                                       ...  
HEAVY                                      1
R/V                                        1
Work                                       1
CASE                                       1
mail                                       1
Name: count, Length: 217, dtype: int64


In [6]:
services = ['abulance', 'almbulance','AMB', 'AMBU', 'AMBUL', 'AMBULACE','AMBULANCE','AMBULENCE', 'fdny ems', 'FDNY AMBUL', 'Leased amb',
             'ambulance', 'FDNY ENGIN','NYC AMBULA', 'fdny ambulance', 'GEN AMBUL', 'FDNY Engin', 'Sanit', 'NYFD', 'Ambulance','FDNY #226', 
             'GEN  AMBUL', 'J1', 'E REVEL SC', 'NYPD', 'FDNY Ambul', 'ambu', 'Ambul', 'ambul', 'Fire Truck', 'Fire Engin', 'Fire truck', 
             'Fire', 'FDNY FIRE', 'FIRE TRUCK', 'FDNY TRUCK', 'FIRTRUCK', 'FDNY fire', 'fire truck', 'FIRE','FIRETRUCK','FIRE ENGIN',
             'FD tr', 'fd tr', 'FD TR', 'fdny', 'FDNY', 'FIRET', 'firet', 'NYC', 'fire', 'government', 'GOVER', 'NYC FD']

car = ['Sedan', 'PK', 'Convertible', 'Station Wagon/Sport Utility Vehicle', 'MOTOR SKAT', 'PICK UP', 'Can',
       '4 dr sedan', 'UNK', 'Pick up tr', 'SUV', 'FLAT', '3-Door', 'SMART CAR', 'Street Cle', '2 dr sedan', 
       'CHEVY EXPR', 'suburban', 'Pick up Tr', 'F150XL PIC', 'Wh Ford co','MINI', 'Motorized Home', 'RV', 'R/V', 
       'Box t', 'Pickup with mounted Camper', 'Subn', 'UHAUL', 'SLINGSHOT', 'UT', 'cross', 
       'Elect', 'Hopper', 'OMR', 'RGS', 'CAMP', 'PICKU', 'ELECT', '2- to']

van = ['Van', 'van', 'ford van', 'Van Camper', 'WORK VAN', 'VAN T', 'Work Van', 'Cargo Van']

truck = ['Tractor Truck Diesel', 'Flat Bed', 'Box Truck', 'Pick-up Truck', 'Dump', 'Concrete Mixer', 'Tanker', 
         'Tractor Truck Gasoline', 'FDNY LADDE', 'Tow Truck / Wrecker', 'Chassis Cab', 
         'Bulk Agriculture', 'FLATBED FR', 'Open Body', 'Flat Rack', 'Armored Truck', 'truck', 'TRAILER', 
         'Lift Boom','BOX TRUCK', 'Cement Tru', 'USPS/GOVT', 'TRUCK VAN', 'UTILITY', 'utility', 'POWER SHOV', 
         'DELIVERY T', 'SWT', 'Trac', 'USPS', 'Beverage Truck', 'Refrigerated Van', 'PSD', 'TRAC', 'Tow Truck', 'COURIER',
         'Courier', 'message si', 'box', 'F550','DELV', 'box truck', 'commercial', 'Tractor tr', 'TRUCK', 
         'Stake or Rack', 'COMMERCIAL', 'dilevery t', 'FREIGHT FL', 'MOVING VAN',  'UPS TRUCK', 'dump truck', 'Freight', 
         'USPS VAN', 'TRUCK FLAT', 'BOBCAT FOR', 'Tractor Tr', 'DELIVERY V', 'DOT EQUIPM', 'Livestock Rack', 'DUMP', '18 WHEELER', 
         'MAIL TRUCK', 'FOOD TRUCK', 'Bucket Tru', 'FLATBED', 'POSTO', 'FREIG', 'DELIV', 'trail', 
         'TRAIL', 'UTILITY VE', 'HEAVY', 'UPS T', 'BACKH', 'Tractor', 'Light Trai', 'Fork lift',  'FORK LIFT', 
         'Dump', 'Utility', 'Pumper', 'Front-Load', 'DRILL RIG', 'MECHANICAL', 'mail', 'Garba', 'TRACT', 
         'Garbage or Refuse', 'GARBAGE TR', 'Trailer', 'trailer', 'UTIL', 'Delv',
         'ROAD SWEEP', 'LIGHT TRAI', 'USPS TRUCK', 'USPS TRUCK', 'usps', 'Semi', 'CEMEN', 'Backh', 'deliv', 'tow', 'dump', 'Elect', 'utili',
         'Util', 'ACCES', 'BOBCA' ,'TANK' ,'TRACK', 'utili', 'FOOD', 'Spc', 'BED T', 'comme', 'PAS', 'SWEEP', 'BOX T', 
         'CASE', 'Work', 'LIBER', 'COMB', 'DUMPS', 'Utili', 'cross', 'gator', 'CAT', 'GARBA', 
         'semi',  'UTILI', 'R/V C', 'sgws', 'Cat 9', 'MACK', 'SPC', 'Enclosed Body - Removable Enclosure', 'delv', 'MAIL', 
         'box t', 'garba', 'CONCR', 'Pallet', 'FED E', 'COMME', 'TRLR', 'LOADE', 'rv', 'Pick', 'NS AM', 'STAK', 'FORKL', 'Tract', 'freig', 
         'Dumps', 'forkl', 'TRK', 'BROOM', 'Trail', 'Glass Rack', 'US POSTAL', 'TRT', 'pas', 'COM',  'CHERR', 'UTV', 'NEW Y', 
         'TOW T', 'tract', 'STREE' ]

bus = ['Bus', 'School Bus', 'bus', 'MTA BUS', 'postal bus', 'MTA b', 'MTA B', 'SCHOO', 'schoo']

taxi = ['Taxi', 'LIMO', 'Pedicab', 'Cab', 'TAXI', 'taxi cab', 'ride service']

two_wheeled = ['E-Scooter', 'Motorbike', 'Motorcycle', 'Bike', 'E-Bike', 'Motorscooter', 'Minibike', 'Scooter', 'E-BIKE', 'scooter', 
                'E-UNICYCLE', 'SKATEBOARD', 'Minicycle', 'moped', 'SCOOTER', 'scoot', 'Scoot', 'e sco', 'E-Bik', 'EBIKE', 'motor', 'E-MOT', 
                'ELEC. UNIC', 'MOTORSCOOT', 'two', 'BICYC', 'ELECTRIC S', 'e-bik', 'bike', 'moto', 'GATOR', 'Motorcycle', 'MOPED', 'WHEEL',
                'E-BIK', 'Wheel', 'SCOOT', 'Moped']

others = ['Golf Cart', 'GOLF CART', 'forlift', 'Carry All', 'Street Swe', 'DELIVERY', 'UNKNOWN', 'Multi-Wheeled Vehicle', 'Fork Lift', 'FORKLIFT',
          'Lawnmower', 'dark color', 'PC', 'Unknown', 'FD LADDER', 'OTH', 'Horse', 'JOHN DEERE', 'TOWER', 'Lunch Wagon', 'WORKH', 
          'WORK', '99999', 'const', 'BLOCK', 'unk', 'CRANE', 'BULLD', 'BK', 'seagr', 'Trc', 'Go kart', 'UNKN', 'Forklift', 'forklift', 
          'backhoe', 'gator','CONST', 'self', 'DEMA-', 'FORK-', '1C', 'Comm', 'SELF', 'fork', 'FORK', 'POWER', 'Sprin', 'FRONT', 'unknown', 
          'MOTOR', 'UNKNO', 'GRAY', np.nan]



In [7]:
df['VEHICLE TYPE'] = df['VEHICLE TYPE'].replace(services, 'SERVICES')
df['VEHICLE TYPE'] = df['VEHICLE TYPE'].replace(van, 'VAN')
df['VEHICLE TYPE'] = df['VEHICLE TYPE'].replace(car, 'CAR')
df['VEHICLE TYPE'] = df['VEHICLE TYPE'].replace(truck, 'TRUCK')
df['VEHICLE TYPE'] = df['VEHICLE TYPE'].replace(bus, 'BUS')
df['VEHICLE TYPE'] = df['VEHICLE TYPE'].replace(taxi, 'TAXI')
df['VEHICLE TYPE'] = df['VEHICLE TYPE'].replace(two_wheeled, 'TWO_WHEELED')
df['VEHICLE TYPE'] = df['VEHICLE TYPE'].replace(others, 'OTHERS')

In [8]:
print(df['VEHICLE TYPE'].value_counts())

VEHICLE TYPE
CAR            65200
TRUCK           5953
TAXI            3822
TWO_WHEELED     1939
BUS             1099
VAN              562
OTHERS           530
SERVICES         278
Name: count, dtype: int64


In [9]:
# missing values percentages for each column
df.isna().sum() / df.shape[0]

CRASH DATE                       0.000000
CRASH TIME                       0.000000
BOROUGH                          0.355252
ZIP CODE                         0.355404
LATITUDE                         0.060517
LONGITUDE                        0.060517
NUMBER OF PEDESTRIANS INJURED    0.000000
NUMBER OF PEDESTRIANS KILLED     0.000000
NUMBER OF CYCLIST INJURED        0.000000
NUMBER OF CYCLIST KILLED         0.000000
NUMBER OF MOTORIST INJURED       0.000000
NUMBER OF MOTORIST KILLED        0.000000
CONTRIBUTING FACTOR              0.002318
VEHICLE TYPE                     0.000000
dtype: float64

Now, since there are a large amount of missing Borough and Zip Code cells (over 28.200), we will infer their value using longitude and latitude coordinates for those rows with missing values.

In [10]:
df['LATITUDE'] = pd.to_numeric(df['LATITUDE'], errors='coerce')
df['LONGITUDE'] = pd.to_numeric(df['LONGITUDE'], errors='coerce')

In [23]:
#pip install pandas geopandas shapely

In [12]:
df['ZIP CODE'] = pd.to_numeric(df['ZIP CODE'], errors='coerce')

In [13]:
import geopandas as gpd
from shapely.wkt import loads
from shapely.geometry import Point

zipcodes_df = pd.read_csv('data/zip_code_area.csv')
boroughs_df = pd.read_csv('data/borough_area.csv')

zipcodes_df['geometry'] = zipcodes_df['the_geom'].apply(loads)
boroughs_df['geometry'] = boroughs_df['the_geom'].apply(loads)

# convert dataframes to GeoDataFrames
zipcodes_gdf = gpd.GeoDataFrame(zipcodes_df, geometry='geometry', crs="EPSG:4326")
boroughs_gdf = gpd.GeoDataFrame(boroughs_df, geometry='geometry', crs="EPSG:4326")

# Function to asses borough and zip code using latitude and longitude as input parameters
def get_borough_zip(lat, lon):
    point = Point(lon, lat)
    
    # Find borough
    borough = None
    for _, row in boroughs_gdf.iterrows():
        if row['geometry'].contains(point):
            borough = row['BoroName']
            break
    
    # Find ZIP code
    zip_code = None
    for _, row in zipcodes_gdf.iterrows():
        if row['geometry'].contains(point):
            zip_code = row['MODZCTA']
            break

    return borough, zip_code


In [14]:
# Apply get_borough_zip function to rows with missing BOROUGH or ZIP CODE values
def complete_locations(data):
    missing = data[data['BOROUGH'].isna() | data['ZIP CODE'].isna()]

    for idx, row in missing.iterrows():
        borough, zip_code = get_borough_zip(row['LATITUDE'], row['LONGITUDE'])
        if borough:
            data.at[idx, 'BOROUGH'] = borough
        if zip_code:
            data.at[idx, 'ZIP CODE'] = zip_code

    return data

In [15]:
df = complete_locations(df)

Now, for remaining rows with no Borough, Zip code, latitude nor longitude, plus the ones to have null latitude or longitude, we are going to discard them as their inclusion would interfere with data visualization.

In [23]:
df = df.dropna(subset=['LATITUDE', 'LONGITUDE', 'BOROUGH', 'ZIP CODE'])
df = df[(df['LATITUDE'] != 0) & (df['LONGITUDE'] != 0)]

In [19]:
# all borough names are set to upper case letters
df['BOROUGH'] = df['BOROUGH'].str.upper()

Following this, we categorize matching contributing factors using clustering techniques

In [24]:
print(df['CONTRIBUTING FACTOR'].value_counts())

CONTRIBUTING FACTOR
Driver Inattention/Distraction                           18493
Unspecified                                              16233
Following Too Closely                                     6774
Failure to Yield Right-of-Way                             5102
Backing Unsafely                                          3847
Passing or Lane Usage Improper                            3329
Passing Too Closely                                       3200
Unsafe Lane Changing                                      2505
Other Vehicular                                           2071
Turning Improperly                                        1838
Traffic Control Disregarded                               1320
Driver Inexperience                                       1223
Reaction to Uninvolved Vehicle                            1221
Unsafe Speed                                              1045
Alcohol Involvement                                        713
View Obstructed/Limited            

In [25]:
Distraction = ['Driver Inattention/Distraction', 'Cell Phone (hand-Held)', 'Passenger Distraction', 'Outside Car Distraction', 
               'Eating or Drinking', 'Using On Board Navigation Device', 'Cell Phone (hands-free)', 'Other Electronic Device', 
               'Texting','Listening/Using Headphones']

Environment = ['Pavement Slippery', 'Pavement Defective', 'Obstruction/Debris', 'Animals Action', 'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion', 'Lane Marking Improper/Inadequate']

Motor_problem = ['Oversized Vehicle', 'Brakes Defective', 'Steering Failure', 'Tire Failure/Inadequate', 'Accelerator Defective', 
                 'Traffic Control Device Improper/Non-Working', 'Tinted Windows', 'Headlights Defective', 'Other Lighting Defects', 
                 'Shoulders Defective/Improper', 'Tow Hitch Defective', 'Windshield Inadequate']

Driving_mistake = ['Unsafe Lane Changing', 'Failure to Yield Right-of-Way', 'Turning Improperly', 'Traffic Control Disregarded', 
                   'Unsafe Speed', 'Passing or Lane Usage Improper', 'Following Too Closely', 'Aggressive Driving/Road Rage', 
                   'Passing Too Closely', 'Backing Unsafely', 'Failure to Keep Right' ]

Driver_related = ['Driver Inexperience', 'Fell Asleep', 'Illnes', 'Fatigued/Drowsy', 
                  'Lost Consciousness', 'Physical Disability', 'Prescription Medication']

Driver_related_illegal = ['Alcohol Involvement', 'Drugs (illegal)']

Other_vehicular = ['Other Vehicular', 'View Obstructed/Limited', 'Reaction to Uninvolved Vehicle', 'Glare']

Uncontrolled = ['Driverless/Runaway Vehicle', 'Vehicle Vandalism']

Unspecified = ['Unspecified', None, np.nan]


df['CONTRIBUTING FACTOR'] = df['CONTRIBUTING FACTOR'].replace(Distraction, 'Driver distraction')
df['CONTRIBUTING FACTOR'] = df['CONTRIBUTING FACTOR'].replace(Environment, 'Roadway issues')
df['CONTRIBUTING FACTOR'] = df['CONTRIBUTING FACTOR'].replace(Motor_problem, 'Vehicle defect')
df['CONTRIBUTING FACTOR'] = df['CONTRIBUTING FACTOR'].replace(Driving_mistake, 'Improper driving and traffic rules violation')
df['CONTRIBUTING FACTOR'] = df['CONTRIBUTING FACTOR'].replace(Driver_related, 'Driver condition')
df['CONTRIBUTING FACTOR'] = df['CONTRIBUTING FACTOR'].replace(Driver_related_illegal, 'Substance use')
df['CONTRIBUTING FACTOR'] = df['CONTRIBUTING FACTOR'].replace(Other_vehicular, 'Vehicle interaction factors')
df['CONTRIBUTING FACTOR'] = df['CONTRIBUTING FACTOR'].replace(Uncontrolled, 'Uncontrolled vehicle')

In [26]:
print(df['CONTRIBUTING FACTOR'].value_counts())

CONTRIBUTING FACTOR
Improper driving and traffic rules violation    29395
Driver distraction                              18970
Unspecified                                     16233
Vehicle interaction factors                      4009
Driver condition                                 1633
Roadway issues                                   1214
Vehicle defect                                   1058
Substance use                                     762
Uncontrolled vehicle                               76
Name: count, dtype: int64


Now, additional columns will be added regarding accidents' date and time to ease the following creation of visualizations. 4 new columns are created: Datetime (with the complete date), Month (with the month as a string), Hour (for the accidents' time), Week_day (with the day of the week as a string) and Day (for the day of the month)

In [28]:
df['CRASH TIME'] = df['CRASH TIME'].apply(lambda x: x if len(x.split(':')) == 3 else x + ':00')

df['DATETIME'] = pd.to_datetime(df['CRASH DATE'] + ' ' + df['CRASH TIME'], format='%m/%d/%Y %H:%M:%S')

df['MONTH'] = df['DATETIME'].dt.strftime('%B')

df['HOUR'] = df['DATETIME'].dt.hour

df['WEEK_DAY'] = df['DATETIME'].dt.strftime('%A')

df['DAY'] = df['DATETIME'].dt.day

Now, dataset with weather conditions will be merged to enrich current dataset with a column for the weather condition for adding context

In [30]:
df_weather = pd.read_csv('data/weather2018.csv')

df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'], format='%m/%d/%Y')
df_weather['datetime'] = pd.to_datetime(df_weather['datetime'], format='%Y-%m-%d')

df_weather.rename(columns={'datetime': 'CRASH DATE'}, inplace=True)

df_crash = pd.merge(df, df_weather[['CRASH DATE', 'icon']], on='CRASH DATE', how='left')

df_crash.rename(columns={'icon': 'WEATHER'}, inplace=True)

Columns regarding casualties will be added, plus a new column stating incidents' severity

In [31]:
# total count of injuries and deaths
df_crash['TOTAL_INJURIES'] = df_crash[['NUMBER OF PEDESTRIANS INJURED', 
                           'NUMBER OF CYCLIST INJURED', 
                           'NUMBER OF MOTORIST INJURED']].sum(axis=1)

df_crash['TOTAL_DEATHS'] = df_crash[['NUMBER OF PEDESTRIANS KILLED', 
                         'NUMBER OF CYCLIST KILLED', 
                         'NUMBER OF MOTORIST KILLED']].sum(axis=1)

In [32]:
df_crash['SEVERITY'] = df_crash.apply(
    lambda row: 'Death' if row['TOTAL_DEATHS'] > 0
    else ('Injury' if row['TOTAL_INJURIES'] > 0 else 'No Damage'),
    axis=1
)

Finally, some additional columns which have been useful to complete data but are no longer of use will be disregarded to have the simplest dataset possible.

In [33]:
df_crash.drop(columns=['CRASH DATE', 'CRASH TIME', 
                       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
                       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
                       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED'], inplace=True)

In [34]:
df_crash.columns

Index(['BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'CONTRIBUTING FACTOR',
       'VEHICLE TYPE', 'DATETIME', 'MONTH', 'HOUR', 'WEEK_DAY', 'DAY',
       'WEATHER', 'TOTAL_INJURIES', 'TOTAL_DEATHS', 'SEVERITY'],
      dtype='object')

Finally, resulting csv is stored as a csv file in the data folder to be used for further analytics

In [35]:
df_crash.to_csv('data/accidents_preprocessed.csv', index=False)